In [1]:
import requests 
import os
from datetime import datetime, timedelta
import json
import pandas as pd
import csv

In [2]:
def get_metadata():
    lines = 10 # 10tk praegu
    with open(f"arxiv-metadata-oai-snapshot.json", 'r') as f:
        for line in f:
            if lines == 0:
                break
            lines -= 1
            yield line

In [22]:
metadata = get_metadata()

authors_parsed = []
categories	 = []
doi = []
ids = []
titles = []
versions = []
author_ids = []
authors = []
for paper in metadata:
    metaDict = json.loads(paper)

    authorsp = metaDict['authors_parsed']
    a_ids = []
    for a in authorsp:
        author_id = "_".join(a)  
        a_ids.append(author_id)    
        authors.append({
                "authorId": author_id, 
                "first_name": a[1],
                "last_name": a[0]
            })

    ids.append(metaDict['id'])
    doi.append(metaDict['doi'])
    titles.append(metaDict['title'])
    authors_parsed.append(metaDict['authors_parsed'])
    author_ids.append(a_ids)
    categories.append(metaDict['categories'])
    versions.append(metaDict['versions'])


df = pd.DataFrame({'id' : ids, 'doi': doi, 'title' : titles, 'authors_parsed': authors_parsed, 'author_ids': author_ids, 'categories' : categories, 'versions': versions})
author_df = pd.DataFrame.from_records(authors)

# Remove the duplicates
df = df[~df.title.duplicated()]

# Enforcing types
df['doi']= df['doi'].astype('str')
df['id']= df['id'].astype('str')
df['title']= df['title'].astype('str')
df['authors_parsed']= df['authors_parsed'].astype('str')
df['categories']= df['categories'].astype('str')
df['versions']= df['versions'].astype('str')

In [27]:
author_df['gender'] = 'Unknown'
author_df # Täisnimi oleks vaja saada Crossref API-st, enamusel pole siin täisnime

,authorId,first_name,last_name,gender
0,Balázs_C._,C.,Balázs,Unknown
1,Berger_E. L._,E. L.,Berger,Unknown
2,Nadolsky_P. M._,P. M.,Nadolsky,Unknown
3,Yuan_C. -P._,C. -P.,Yuan,Unknown
4,Streinu_Ileana_,Ileana,Streinu,Unknown
5,Theran_Louis_,Louis,Theran,Unknown
6,Pan_Hongjun_,Hongjun,Pan,Unknown
7,Callan_David_,David,Callan,Unknown
8,Abu-Shammala_Wael_,Wael,Abu-Shammala,Unknown
9,Torchinsky_Alberto_,Alberto,Torchinsky,Unknown


In [28]:
# Võtan ühe naise, kellel eesnimi olemas
author_df.loc[4]['first_name']

'Ileana'

In [33]:
first_name = author_df.loc[4]['first_name']
last_name = author_df.loc[4]['last_name']
r = requests.get(url=f'https://innovaapi.aminer.cn/tools/v1/predict/gender?name={first_name}+{last_name}&org=')

In [34]:
print(r.json())

{'status': 0, 'message': 'success', 'data': {'FGNL': {'gender': 'female', 'probability': '100.00%'}, 'WebGP': {'gender': 'UNKNOWN', 'probability': ''}, 'Face': {'gender': 'UNKNOWN', 'probability': ''}, 'Final': {'gender': 'female', 'm_proba': '0.00%', 'f_proba': '100.00%'}}}


In [36]:
data = r.json()
gender = data['data']['Final']['gender']

In [39]:
gender

'female'

In [38]:
author_df.loc[4]['gender'] = gender

In [40]:
author_df.head()

,authorId,first_name,last_name,gender
0,Balázs_C._,C.,Balázs,Unknown
1,Berger_E. L._,E. L.,Berger,Unknown
2,Nadolsky_P. M._,P. M.,Nadolsky,Unknown
3,Yuan_C. -P._,C. -P.,Yuan,Unknown
4,Streinu_Ileana_,Ileana,Streinu,female
